Monitorear el entrenamiento con tensorboard (tensorboard --logdir=runs) en el directorio.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import os
import numpy as np
from torch.utils.tensorboard import SummaryWriter

os.environ['KMP_DUPLICATE_LIB_OK']='True'

torch.cuda.empty_cache()

In [ ]:
# Definir la arquitectura de la CNN
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1) # Primera capa de convolución
        self.conv1_bn = nn.BatchNorm2d(32) # Capa de normalización de lote luego de cada capa de convolución
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) # Segunda capa de convolución
        self.conv2_bn = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3_bn = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 500)
        self.fc2 = nn.Linear(500, 10)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1_bn(self.conv1(x))))
        x = self.pool(F.relu(self.conv2_bn(self.conv2(x))))
        x = self.pool(F.relu(self.conv3_bn(self.conv3(x))))
        x = x.view(-1, 128 * 4 * 4)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
def main():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # Inicializar el escritor de TensorBoard
    writer = SummaryWriter('runs/experiment_1')
    # Cargar y normalizar los conjuntos de datos CIFAR10

    transform = transforms.Compose(
        [transforms.RandomHorizontalFlip(),  # Aumento de datos con flip
        transforms.RandomCrop(32, padding=4),  # Aumento de datos con crop
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    full_trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)

    train_size = int(0.8 * len(full_trainset))
    test_size = len(full_trainset) - train_size
    trainset, valset = torch.utils.data.random_split(full_trainset, [train_size, test_size])

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                            shuffle=True, num_workers=4)
    valloader = torch.utils.data.DataLoader(valset, batch_size=32,
                                            shuffle=True, num_workers=4)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                            shuffle=False, num_workers=4)

    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    # Crear la red
    net = Net()
    # Mover el modelo a la GPU
    net.to(device)
    
    # Definir la función de pérdida y el optimizador
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)

    # Cargar el último checkpoint si existe
    start_epoch = 0
    if os.path.exists('checkpoints/latest_checkpoint.pth'):
        checkpoint = torch.load('checkpoints/latest_checkpoint.pth')
        net.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        print(f'Loaded checkpoint from epoch {start_epoch}')
    else:
        print("No checkpoint found, starting from scratch.")

    # Entrenar la red y guardar las pérdidas para la visualización
    train_losses = []
    val_losses = []
    best_val_loss = float("inf")
    epochs_no_improve = 0
    n_epochs_stop = 15

    for epoch in range(start_epoch,20000):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            # move the inputs and labels to GPU
            inputs, labels = data[0].to(device), data[1].to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # print statistics
            running_loss += loss.item()
            if i % 100 == 99 or i == len(trainloader) - 1:  # Asume que la indexación comienza en 0
                avg_loss = running_loss / (i+1)
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, avg_loss))
                train_losses.append(avg_loss)
                writer.add_scalar('training loss', avg_loss, epoch * len(trainloader) + i)
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for data in valloader:
                images, labels = data[0].to(device), data[1].to(device)
                outputs = net(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        avg_val_loss = val_loss / total
        val_accuracy = correct / total
        print(f'Validation loss: {avg_val_loss:.3f}, Validation accuracy: {val_accuracy:.3f}')
        val_losses.append(avg_val_loss)
        
        if epoch % 10 == 9:
            torch.save({
                'epoch': epoch,
                'model_state_dict': net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss
            }, f'checkpoints/checkpoint{epoch}.pth')
            print(f'Saved checkpoint at epoch {epoch}')

        # Guardar el último checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
        }, 'checkpoints/latest_checkpoint.pth')
        
        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(net.state_dict(), 'model.pth')
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve == n_epochs_stop:
                print('Early stopping!')
                # Load the best state dict
                net.load_state_dict(torch.load('model.pth'))
                # Exit the loop
                break

    print('Finished Training')
    writer.close()

    import matplotlib.pyplot as plt

    # Visualizar la pérdida durante el entrenamiento
    plt.plot(train_losses)
    plt.xlabel('Mini-batch')
    plt.ylabel('Loss')
    plt.show()

    # Evaluación del modelo en el conjunto de datos de prueba
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

    # Guardar el modelo entrenado
    torch.save(net.state_dict(), 'model.pth')


if __name__ == "__main__":
    main()

In [ ]:
# Cargar el modelo entrenado
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = Net()
net.load_state_dict(torch.load('model.pth'))
net.to(device)
net.eval()  # poner el modelo en modo de evaluación

In [ ]:
# Cargar y normalizar el conjunto de datos de prueba CIFAR10
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=10,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# Obtener algunas imágenes de prueba aleatorias
images, labels = next(iter(testloader))

# Mostrar imágenes
imshow(torchvision.utils.make_grid(images))
# Imprimir etiquetas verdaderas
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(10)))
images, labels = images.to(device), labels.to(device)
outputs = net(images)

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(10)))